In [1]:
import KillerEnv as env
import random
import numpy as np
from IPython.display import clear_output

Killer Functions Imported


In [2]:
episodes = 50

gamma = 1

done = False

In [3]:
action_space = 14
obs_space = 106496

# We need a Q table for each player
Q1 = np.zeros([obs_space, action_space])
Q2 = np.zeros([obs_space, action_space])
Q3 = np.zeros([obs_space, action_space])
Q4 = np.zeros([obs_space, action_space])

# We also need how many times each player has visited a state
state_count = np.zeros([obs_space, 4])

# Finally, while playing the game, each player should remember each state and action pair they took
# If the player wins, all those state, action pairs will get +1 in that player's Q-table
# If the player loses, all those state, action pairs will get -1 in that player's Q-table
game_record1 = []
game_record2 = []
game_record3 = []
game_record4 = []

winner = 1

In [45]:
def get_epsilon(N_state_count, N_zero=10):
    # This will determine how random our agent should act given how many times it has visited this state
    
    return N_zero / (N_zero + N_state_count)

In [5]:
def get_best_action(Q, state, action_size, available_tiers):
    # Will either choose a random action or the best action in this state's row of the Q state
    
    best_action = np.argmax(Q[state])
    
    return best_action

In [6]:
def step_player(player=env.turn+1):
    # Figure out what to play
    state = env.obs_state(player)
    global done
    global game_record1, game_record2, game_record3, game_record4
    global winner

    #action = get_action(Q[player], state, state_count[state][player-1], action_space, available_tiers) 
    #print(action)
    
    epsilon = get_epsilon(state_count[state, player-1])
    action_type = np.random.choice(["random", "best"], p=[1.0 - epsilon, epsilon])
    
    if action_type == "best":
        if player == 4:
            tierlist = env.tier_moves(env.p4_original, env.get_moves(env.p4))
            best_action = np.argmax(Q4[state])
            # if the best action is passing...
            if best_action == 13:
                move = [0]
                env.play(env.p4, [0])
            else:
                move = env.tier_to_move(best_action , env.p4, env.p4_original)
                env.play(env.p4, move)
            # Record the state and action pair
            game_record4 += [[state, env.move_to_tier(move, env.p4_original)]]
            # Check if there's a winner
            if len(env.p4) == 0:
                print("Player 4 has won!!!")
                winner = 4
                done = True
        elif player == 3:
            tierlist = env.tier_moves(env.p3_original, env.get_moves(env.p3))
            best_action = np.argmax(Q3[state])
            # if the best action is passing...
            if best_action == 13:
                move = [0]
                env.play(env.p3, [0])
            else:
                move = env.tier_to_move(best_action , env.p3, env.p3_original)
                env.play(env.p3, move)
            # Record the state and action pair
            game_record3 += [[state, env.move_to_tier(move, env.p3_original)]]
            # Check if there's a winner
            if len(env.p3) == 0:
                winner = 3
                print("Player 3 has won!!!")
                done = True
        elif player == 2:
            tierlist = env.tier_moves(env.p2_original, env.get_moves(env.p2))
            best_action = np.argmax(Q2[state])
            # if the best action is passing...
            if best_action == 13:
                move = [0]
                env.play(env.p2, [0])
            else:
                move = env.tier_to_move(best_action , env.p2, env.p2_original)
                env.play(env.p2, move)
            # Record the state and action pair
            game_record2 += [[state, env.move_to_tier(move, env.p2_original)]]
            if len(env.p2) == 0:
                winner = 2
                print("Player 2 has won!!!")
                done = True
        else:
            tierlist = env.tier_moves(env.p1_original, env.get_moves(env.p1))
            best_action = np.argmax(Q1[state])
            # if the best action is passing...
            if best_action == 13:
                move = [0]
                env.play(env.p1, [0])
            else:
                move = env.tier_to_move(best_action , env.p1, env.p1_original)
                env.play(env.p1, move)
            # Record the state and action pair
            game_record1 += [[state, env.move_to_tier(move, env.p1_original)]]
            if len(env.p1) == 0:
                winner = 1
                print("Player 1 has won!!!")
                done = True
    
    else:
        if player == 4:
            move = random.choice(env.get_moves(env.p4))
            env.play(env.p4, move)
            # Record state and move
            game_record4 += [[state, env.move_to_tier(move, env.p4_original)]]
            # Check if there's a winner
            if len(env.p4) == 0:
                winner = 4
                print("Player 4 has won!!!")
                done = True
        elif player == 3:
            move = random.choice(env.get_moves(env.p3))
            env.play(env.p3, move)
            # Record state and move
            game_record3 += [[state, env.move_to_tier(move, env.p3_original)]]
            # Check if there's a winner
            if len(env.p3) == 0:
                winner = 3
                print("Player 3 has won!!!")
                done = True
        elif player == 2:
            move = random.choice(env.get_moves(env.p2))
            env.play(env.p2, move)
            # Record state and move
            game_record2 += [[state, env.move_to_tier(move, env.p2_original)]]
            if len(env.p2) == 0:
                winner = 2
                print("Player 2 has won!!!")
                done = True
        else:
            move = random.choice(env.get_moves(env.p1))
            env.play(env.p1, move)
            # Record state and move
            game_record1 += [[state, env.move_to_tier(move, env.p1_original)]]
            if len(env.p1) == 0:
                winner = 1
                print("Player 1 has won!!!")
                done = True
                
    # Add one to the state count matrix for this state and player
    state_count[state, player-1] += 1 

In [15]:
def record_wins():
    if winner == 4:
        for s_a_pair in game_record4:
            Q4[s_a_pair[0], s_a_pair[1]] += 1
        for s_a_pair in game_record3:
            Q3[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record2:
            Q2[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record1:
            Q1[s_a_pair[0], s_a_pair[1]] += -1
    elif winner == 3:
        for s_a_pair in game_record4:
            Q4[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record3:
            Q3[s_a_pair[0], s_a_pair[1]] += 1
        for s_a_pair in game_record2:
            Q2[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record1:
            Q1[s_a_pair[0], s_a_pair[1]] += -1
    elif winner == 2:
        for s_a_pair in game_record4:
            Q4[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record3:
            Q3[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record2:
            Q2[s_a_pair[0], s_a_pair[1]] += 1
        for s_a_pair in game_record1:
            Q1[s_a_pair[0], s_a_pair[1]] += -1
    else:
        for s_a_pair in game_record4:
            Q4[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record3:
            Q3[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record2:
            Q2[s_a_pair[0], s_a_pair[1]] += -1
        for s_a_pair in game_record1:
            Q1[s_a_pair[0], s_a_pair[1]] += 1
    print("Wins recorded.")

In [16]:
def step_game():
    env.newgame()
    global game_record1, game_record2, game_record3, game_record4
    game_record1, game_record2, game_record3, game_record4 = [], [], [], []
    global done
    done = False
    
    if 3.0 in env.p1:
        order = [1, 2, 3, 4]
    elif 3.0 in env.p2:
        order = [2, 3, 4, 1]
    elif 3.0 in env.p3:
        order = [3, 4, 1, 2]
    elif 3.0 in env.p4:
        order = [4, 1, 2, 3]
    else:
        order = [1, 2, 3, 4]
    
    while done == False:
        for i in order:
            step_player(i)
            #env.render_game()
            if done == True:
                break
    
    record_wins()

In [11]:
step_game()

P4 plays  [8.25]
P1 plays  [11.25, 12.75, 13.5, 14.25, 15.25]
P2 plays  [9.75, 10.5, 11.0, 12.0]
P4 plays  [7.0, 7.5]
P3 plays  [15.5, 15.75]
P4 plays  [6.5, 7.25, 8.5]
P1 plays  [5.25]
P3 plays  [5.0, 6.0, 7.75]
P3 plays  [14.0]
P2 plays  [3.0, 3.75]
P3 plays  [10.75]
P1 plays  [8.0, 8.75]
P4 plays  [5.5]
P4 plays  [11.75]
P2 plays  [13.75]
P1 plays  [6.25]
P2 plays  [9.5]
P3 plays  [3.5]
P4 plays  [4.75]
P2 plays  [4.5]
P3 plays  [13.0]
P4 plays  [12.25]
P4 plays  [11.5]
P2 plays  [15.0]
P2 plays  [4.25]
P3 plays  [9.0]
P1 plays  [13.25]
P4 plays  [14.75]
P1 plays  [6.75]
P1 plays  [3.25]
P2 plays  [14.5]
P1 plays  [5.75]
Player 1 has won!!!
Wins recorded.


In [19]:
def evaluate_policy(episodes=50):
    # Gets the win ratio by playing 10000 games

    wins = [0, 0, 0, 0]
    for e in range(episodes):
        step_game()
        wins[winner-1] += 1
        
    return [i/episodes for i in wins]


In [54]:
def step_player_vs_random(player=env.turn+1):
    # Figure out what to play
    state = env.obs_state(player)
    global done
    global game_record1, game_record2, game_record3, game_record4
    global winner

    #action = get_action(Q[player], state, state_count[state][player-1], action_space, available_tiers) 
    #print(action)
    
    epsilon = get_epsilon(state_count[state, player-1])
    action_type = np.random.choice(["random", "best"], p=[1.0 - epsilon, epsilon])
    
    
    if player == 4:
            if action_type == "best":
                tierlist = env.tier_moves(env.p4_original, env.get_moves(env.p4))
                best_action = np.argmax(Q4[state])
                # if the best action is passing...
                if best_action == 13:
                    move = [0]
                    env.play(env.p4, [0])
                else:
                    move = env.tier_to_move(best_action , env.p4, env.p4_original)
                    env.play(env.p4, move)
                # Record the state and action pair
                game_record4 += [[state, env.move_to_tier(move, env.p4_original)]]
                # Check if there's a winner
                if len(env.p4) == 0:
                    print("Player 4 has won!!!")
                    winner = 4
                    done = True
            else:
                move = random.choice(env.get_moves(env.p4))
                env.play(env.p4, move)
                # Record state and move
                game_record4 += [[state, env.move_to_tier(move, env.p4_original)]]
                # Check if there's a winner
                if len(env.p4) == 0:
                    winner = 4
                    print("Player 4 has won!!!")
                    done = True
    
    else:
        if player == 3:
            move = random.choice(env.get_moves(env.p3))
            env.play(env.p3, move)
            # Record state and move
            game_record3 += [[state, env.move_to_tier(move, env.p3_original)]]
            # Check if there's a winner
            if len(env.p3) == 0:
                winner = 3
                print("Player 3 has won!!!")
                done = True
        elif player == 2:
            move = random.choice(env.get_moves(env.p2))
            env.play(env.p2, move)
            # Record state and move
            game_record2 += [[state, env.move_to_tier(move, env.p2_original)]]
            if len(env.p2) == 0:
                winner = 2
                print("Player 2 has won!!!")
                done = True
        elif player == 1:
            move = random.choice(env.get_moves(env.p1))
            env.play(env.p1, move)
            # Record state and move
            game_record1 += [[state, env.move_to_tier(move, env.p1_original)]]
            if len(env.p1) == 0:
                winner = 1
                print("Player 1 has won!!!")
                done = True
                
    # Add one to the state count matrix for this state and player
    state_count[state, player-1] += 1 

In [41]:
def step_game_vs_random():
    env.newgame()
    global game_record1, game_record2, game_record3, game_record4
    game_record1, game_record2, game_record3, game_record4 = [], [], [], []
    global done
    done = False
    
    if 3.0 in env.p1:
        order = [1, 2, 3, 4]
    elif 3.0 in env.p2:
        order = [2, 3, 4, 1]
    elif 3.0 in env.p3:
        order = [3, 4, 1, 2]
    elif 3.0 in env.p4:
        order = [4, 1, 2, 3]
    else:
        order = [1, 2, 3, 4]
    
    while done == False:
        for i in order:
            step_player_vs_random(i)
            #env.render_game()
            if done == True:
                break
    
    record_wins()

In [42]:
def evaluate_policy_vs_random(episodes=50):
    # Gets the win ratio by playing 10000 games

    wins = [0, 0, 0, 0]
    for e in range(episodes):
        step_game_vs_random()
        wins[winner-1] += 1
        
    return [i/episodes for i in wins]

In [61]:
p4_wins = [0, 0, 0, 0, 0 , 0 ,0 ,0 ,0, 0]

In [62]:
for i in range(10):
    p4_wins[i] = evaluate_policy_vs_random(150)[3]

P3 plays  [4.5]
P4 plays  [11.75]
P1 plays  [13.25]
P2 plays  [15.25]
P3 plays  [15.75]
P3 plays  [11.25, 12.75, 13.0]
P3 plays  [12.0]
P4 plays  [14.5]
P3 plays  [14.75]
P1 plays  [15.5]
P1 plays  [3.5, 4.75, 5.75, 6.25, 7.25]
P1 plays  [10.75]
P3 plays  [13.75]
P4 plays  [15.0]
P4 plays  [8.0, 8.25, 8.75]
P2 plays  [10.0, 10.25, 10.5]
P3 plays  [3.0]
P4 plays  [13.5]
P4 plays  [7.75]
P1 plays  [12.5]
P1 plays  [9.5]
P2 plays  [12.25]
P2 plays  [6.0, 6.5]
P3 plays  [9.25, 9.75]
P1 plays  [14.0, 14.25]
P2 plays  [11.0, 11.5]
P2 plays  [4.25]
P2 plays  [7.0]
P2 plays  [7.5]
P4 plays  [9.0]
P4 plays  [4.0]
P3 plays  [5.5]
P1 plays  [6.75]
Player 1 has won!!!
Wins recorded.
P4 plays  [10.75]
P1 plays  [14.25]
P3 plays  [14.5]
P4 plays  [15.75]
P4 plays  [5.25, 6.0, 7.0]
P1 plays  [10.25, 11.25, 12.25]
P2 plays  [12.75, 13.25, 14.75]
P2 plays  [9.25]
P3 plays  [15.5]
P3 plays  [4.0, 4.25, 4.75]
P1 plays  [5.0, 5.5, 5.75]
P1 plays  [10.0, 11.5, 12.5]
P1 plays  [8.25]
P2 plays  [12.0]
P3 pla

P2 plays  [13.25, 13.5]
P2 plays  [11.0]
P3 plays  [14.0]
P1 plays  [14.25]
P4 plays  [15.25]
P4 plays  [12.25, 12.5, 12.75]
P4 plays  [3.0]
P1 plays  [9.25]
P2 plays  [11.5]
P3 plays  [11.75]
P4 plays  [15.0]
P4 plays  [9.75]
P1 plays  [14.75]
P1 plays  [10.75]
P1 plays  [5.0]
P2 plays  [5.75]
P3 plays  [7.5]
P4 plays  [13.0]
P1 plays  [13.75]
P1 plays  [3.75]
P2 plays  [10.5]
P2 plays  [8.0]
P4 plays  [8.5]
P1 plays  [8.75]
P2 plays  [9.0]
P3 plays  [9.5]
P2 plays  [12.0]
P2 plays  [8.25]
P3 plays  [10.0]
P2 plays  [11.25]
P3 plays  [7.0, 7.25]
P1 plays  [3.25]
P3 plays  [5.25]
P3 plays  [7.75]
P4 plays  [10.25]
P4 plays  [5.5]
P4 plays  [6.25]
Player 4 has won!!!
Wins recorded.
P1 plays  [10.5, 11.25, 12.25]
P2 plays  [12.75, 13.75, 14.0]
P3 plays  [12.5, 13.0, 14.25]
P3 plays  [3.75]
P4 plays  [5.0]
P2 plays  [13.25]
P1 plays  [14.75]
P3 plays  [15.0]
P3 plays  [8.5, 9.25, 10.0, 11.0]
P4 plays  [8.25, 9.0, 10.75, 11.75]
P4 plays  [15.5, 15.75]
P4 plays  [4.0, 5.75, 6.25]
P4 plays  

P2 plays  [10.0, 11.5, 12.25, 13.5]
P2 plays  [7.5]
P3 plays  [14.25]
P1 plays  [15.25]
P3 plays  [15.75]
P3 plays  [14.0]
P3 plays  [14.5]
P1 plays  [14.75]
P3 plays  [15.0]
P3 plays  [10.25, 11.75, 12.75]
P3 plays  [10.5]
P1 plays  [12.0]
P2 plays  [13.0]
P4 plays  [13.25]
P2 plays  [13.75]
P2 plays  [4.25, 4.5]
P3 plays  [5.25, 5.5]
P1 plays  [9.5, 9.75]
P1 plays  [8.75]
P2 plays  [11.0]
P1 plays  [15.5]
P1 plays  [9.25]
P2 plays  [3.75]
P3 plays  [7.25]
P4 plays  [4.0, 5.75, 6.0]
P1 plays  [5.0, 6.25, 7.0]
P3 plays  [6.75]
Player 3 has won!!!
Wins recorded.
P1 plays  [8.0, 8.25]
P2 plays  [15.25, 15.75]
P2 plays  [9.0, 10.0, 11.25]
P3 plays  [12.25, 13.75, 14.0]
P4 plays  [12.0, 13.5, 14.5]
P4 plays  [12.5]
P1 plays  [14.25]
P2 plays  [14.75]
P3 plays  [15.5]
P3 plays  [9.25, 9.5]
P3 plays  [11.75]
P3 plays  [3.75, 4.75, 5.75]
P2 plays  [4.5, 5.0, 6.0]
P4 plays  [5.5, 6.75, 7.0]
P1 plays  [9.75, 10.75, 11.5]
P1 plays  [8.5]
P2 plays  [11.0]
P4 plays  [12.75]
P1 plays  [13.0]
P1 pla

P1 plays  [5.75]
P2 plays  [12.75]
P3 plays  [15.75]
P1 plays  [6.5]
P2 plays  [8.75]
P3 plays  [12.5]
P1 plays  [15.5]
P1 plays  [6.0, 6.25]
P3 plays  [9.25, 9.75]
P3 plays  [3.25]
P4 plays  [5.0]
P1 plays  [13.75]
P1 plays  [4.0, 4.25]
P2 plays  [5.25, 5.5]
P1 plays  [14.25, 14.5]
Player 1 has won!!!
Wins recorded.
P3 plays  [6.75]
P1 plays  [11.75]
P2 plays  [12.5]
P3 plays  [13.75]
P4 plays  [15.5]
P4 plays  [3.25, 4.5, 5.75, 6.5, 7.25]
P3 plays  [6.25, 7.0, 8.5, 9.0, 10.75]
P3 plays  [9.25]
P4 plays  [14.5]
P1 plays  [15.75]
P2 plays  [14.75]
P2 plays  [4.75]
P3 plays  [13.0]
P1 plays  [14.25]
P1 plays  [7.5]
P3 plays  [11.25]
P4 plays  [9.5]
P1 plays  [9.75]
P1 plays  [3.75]
P2 plays  [8.75]
P4 plays  [10.25]
P1 plays  [13.5]
P2 plays  [14.0]
P3 plays  [3.0]
P1 plays  [15.25]
P1 plays  [12.0, 12.25, 12.75]
P1 plays  [4.0]
P2 plays  [4.25]
P1 plays  [5.5]
Player 1 has won!!!
Wins recorded.
P1 plays  [3.0, 3.5, 3.75]
P3 plays  [15.25, 15.5, 15.75]
P3 plays  [6.0]
P4 plays  [10.75]


P2 plays  [6.0]
P3 plays  [8.25]
P1 plays  [12.5]
P1 plays  [8.0]
P2 plays  [11.5]
P2 plays  [10.25]
P3 plays  [14.5]
P3 plays  [7.75]
P4 plays  [9.25]
P3 plays  [9.75]
P4 plays  [10.5]
P1 plays  [11.25]
P1 plays  [6.75, 7.5, 8.5, 9.0, 10.0]
P3 plays  [5.25, 5.5]
P3 plays  [3.75]
P1 plays  [14.75]
P1 plays  [4.75]
P3 plays  [5.0]
P4 plays  [7.0]
P1 plays  [3.25, 3.5]
P3 plays  [4.5]
P2 plays  [6.5]
Player 2 has won!!!
Wins recorded.
P2 plays  [5.75]
P4 plays  [6.25]
P1 plays  [12.25]
P2 plays  [15.0]
P3 plays  [15.75]
P3 plays  [7.0]
P4 plays  [14.0]
P2 plays  [15.25]
P2 plays  [9.0, 10.25, 11.75]
P2 plays  [10.5]
P3 plays  [11.0]
P4 plays  [12.75]
P1 plays  [13.75]
P2 plays  [14.5]
P3 plays  [14.75]
P3 plays  [6.0, 7.25, 8.75]
P3 plays  [6.5, 7.5, 8.5]
P3 plays  [14.25]
P1 plays  [15.5]
P1 plays  [9.25]
P4 plays  [11.25]
P1 plays  [12.5]
P4 plays  [13.5]
P4 plays  [3.5, 4.5, 5.25]
P1 plays  [8.25, 9.5, 10.75]
P1 plays  [3.25, 4.75, 5.5]
P1 plays  [9.75]
P3 plays  [10.0]
P4 plays  [13.

...

P2 plays  [11.5]
P3 plays  [13.5]
P4 plays  [15.0]
P1 plays  [15.25]
P4 plays  [15.5]
P4 plays  [14.25]
P1 plays  [15.75]
P1 plays  [11.0, 11.25]
P2 plays  [12.25, 12.75]
P2 plays  [7.0]
P3 plays  [7.25]
P4 plays  [9.75]
P1 plays  [10.0]
P2 plays  [14.5]
P2 plays  [3.5]
P3 plays  [11.75]
P4 plays  [13.0]
P3 plays  [14.75]
P3 plays  [3.25, 4.75, 5.25]
P1 plays  [7.5, 8.75, 9.5]
P3 plays  [8.0, 9.25, 10.75]
P1 plays  [7.75]
P2 plays  [8.25]
P4 plays  [12.5]
P4 plays  [6.25]
P1 plays  [14.0]
P1 plays  [10.25]
P1 plays  [13.25, 13.75]
Player 1 has won!!!
Wins recorded.
P2 plays  [10.0, 10.25, 10.5]
P2 plays  [7.0]
P3 plays  [12.25]
P4 plays  [15.75]
P4 plays  [3.5, 4.75, 5.75]
P1 plays  [12.0, 13.0, 14.0]
P3 plays  [13.5, 14.25, 15.5]
P3 plays  [13.25]
P4 plays  [14.5]
P2 plays  [15.0]
P2 plays  [5.25, 6.25, 7.75, 8.75]
P2 plays  [3.0]
P4 plays  [3.25]
P1 plays  [9.0]
P2 plays  [13.75]
P3 plays  [11.5]
P4 plays  [12.5]
P4 plays  [10.75]
P1 plays  [14.75]
P2 plays  [15.25]
P2 plays  [7.25]


P1 plays  [6.75, 7.25, 8.0]
P2 plays  [6.5, 7.0, 8.25]
P2 plays  [12.25]
P3 plays  [14.5]
P4 plays  [15.5]
P4 plays  [3.0, 4.0, 5.0]
P2 plays  [8.5, 9.5, 10.5]
P3 plays  [10.25, 11.0, 12.0]
P3 plays  [4.5, 5.25, 6.25]
P3 plays  [14.25]
P2 plays  [14.75]
P1 plays  [15.25]
P3 plays  [15.75]
P3 plays  [7.5, 7.75]
P4 plays  [3.25]
P1 plays  [4.75]
P2 plays  [9.0]
P4 plays  [9.25]
P2 plays  [9.75]
P4 plays  [11.5]
P2 plays  [13.25]
P1 plays  [15.0]
P1 plays  [10.0]
P2 plays  [3.5]
P4 plays  [8.75]
P1 plays  [5.5]
P4 plays  [13.5]
Player 4 has won!!!
Wins recorded.
P1 plays  [6.5, 7.5, 8.0]
P2 plays  [13.0, 14.5, 15.0]
P2 plays  [14.25]
P4 plays  [15.25]
P1 plays  [15.75]
P1 plays  [9.75]
P2 plays  [10.25]
P3 plays  [11.0]
P3 plays  [3.75, 4.0, 5.25, 6.25]
P3 plays  [6.75]
P4 plays  [9.5]
P2 plays  [13.25]
P3 plays  [13.75]
P3 plays  [6.0]
P1 plays  [10.5]
P2 plays  [12.0]
P3 plays  [14.75]
P3 plays  [12.75]
P2 plays  [14.0]
P4 plays  [15.5]
P4 plays  [7.0, 7.75]
P3 plays  [11.25, 11.75]
P4 

P4 plays  [15.75]
P4 plays  [6.25]
P1 plays  [9.75]
P2 plays  [13.75]
P1 plays  [15.0]
P3 plays  [4.0, 5.75, 6.5]
P3 plays  [9.0]
P3 plays  [5.5]
P4 plays  [8.5]
P3 plays  [9.5]
P3 plays  [7.5]
P2 plays  [8.25]
P3 plays  [11.5]
P3 plays  [13.5]
P3 plays  [3.25, 3.75]
Player 3 has won!!!
Wins recorded.
P1 plays  [8.75]
P2 plays  [13.0]
P3 plays  [15.75]
P3 plays  [10.5, 11.5, 12.75, 13.5, 14.5]
P2 plays  [10.75, 11.75, 12.0, 13.25, 14.75]
P2 plays  [3.75, 4.75, 5.0]
P3 plays  [5.25, 6.25, 7.5]
P4 plays  [6.0, 7.25, 8.25]
P2 plays  [8.5, 9.5, 10.0]
P3 plays  [6.5]
P1 plays  [9.75]
P2 plays  [12.5]
Player 2 has won!!!
Wins recorded.
P2 plays  [13.0, 13.5, 13.75]
P2 plays  [9.75]
P3 plays  [11.75]
P4 plays  [15.75]
P4 plays  [5.25, 5.75]
P1 plays  [14.25, 14.75]
P1 plays  [6.0, 7.75, 8.75]
P1 plays  [6.5]
P3 plays  [14.0]
P1 plays  [15.0]
P1 plays  [15.25]
P2 plays  [15.5]
P3 plays  [7.0, 7.5]
P3 plays  [9.0, 9.25]
P4 plays  [10.0, 10.25]
P4 plays  [3.5, 4.0, 5.0, 6.25]
P1 plays  [12.25]
P

P3 plays  [14.0, 14.25]
P3 plays  [8.0, 8.25, 8.5]
P3 plays  [9.0, 9.5]
P1 plays  [10.5, 10.75]
P2 plays  [12.5, 12.75]
P2 plays  [5.5, 6.5, 7.0]
P1 plays  [6.0, 7.75, 8.75]
P2 plays  [9.75, 10.25, 11.5]
P2 plays  [3.25, 4.25, 5.75]
P4 plays  [9.25, 10.0, 11.0]
P4 plays  [3.0]
P1 plays  [5.25]
P3 plays  [11.25]
P1 plays  [13.5]
P4 plays  [15.25]
P1 plays  [6.25]
P2 plays  [13.75]
Player 2 has won!!!
Wins recorded.
P4 plays  [4.25]
P1 plays  [7.5]
P2 plays  [15.5]
P2 plays  [13.75]
P2 plays  [3.5, 4.5, 5.75]
P3 plays  [9.5, 10.5, 11.5]
P4 plays  [9.0, 10.25, 11.75]
P3 plays  [10.75, 11.25, 12.0]
P1 plays  [12.75, 13.25, 14.5]
P1 plays  [13.0]
P3 plays  [14.75]
P2 plays  [15.0]
P2 plays  [8.75]
P3 plays  [11.0]
P2 plays  [15.75]
P2 plays  [4.0]
P3 plays  [4.75]
P4 plays  [8.5]
P2 plays  [9.25]
P3 plays  [13.5]
P4 plays  [10.0]
P2 plays  [12.5]
P4 plays  [14.0]
P4 plays  [3.75]
P1 plays  [7.25]
P1 plays  [6.5, 7.0, 8.0]
P2 plays  [12.25]
P3 plays  [14.25]
P3 plays  [5.25]
P4 plays  [15.25

P1 plays  [7.5, 8.25, 9.0, 10.75, 11.25]
P1 plays  [4.5]
P2 plays  [11.0]
P3 plays  [13.0]
P2 plays  [15.75]
P2 plays  [5.75, 6.5, 7.25]
P2 plays  [15.5]
P3 plays  [11.75]
P1 plays  [15.0]
P1 plays  [6.25]
Player 1 has won!!!
Wins recorded.
